<a href="https://colab.research.google.com/github/Tiabet/DACON_WebClick/blob/main/WebClick_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
train = pd.read_parquet('/content/drive/MyDrive/data/train.parquet')
test =  pd.read_parquet('/content/drive/MyDrive/data/test.parquet')

In [4]:
# 동일한 결과 보장을 위해 Seed값을 고정합니다
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed를 42로 고정

In [5]:
test.drop('ID', axis = 1, inplace = True)
train.drop('ID', axis = 1, inplace = True)
train.head()

,Click,F01,F02,F03,F04,F05,F06,F07,F08,F09,...,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39
0,1,NSLHFNS,AVKQTCL,DTZFPRW,114.0,ISVXFVA,1,PQZBVMG,LPYPUNA,IZYJZDA,...,NZGEZLW,GTISJWW,380.0,2.0,AXQFZWC,IRUDRFB,NaN,TFJMLCZ,0.0,AURZYDY
1,0,VGIVWZQ,LSUSMVO,PQGWFJZ,26.0,NFRVLWS,43,IMPIGJT,MIGYEEG,NGODWIN,...,NZGEZLW,GTISJWW,466.0,1.0,DRVVDHZ,IRUDRFB,19.0,AUGTURV,0.0,LUZRMLU
2,0,JCDXFYU,PILDDJU,IAGJDOH,119.0,LFPUEOV,0,FFUTIRZ,OFKQGTY,BEZTQIO,...,VHXETCF,KHZNEZF,197.0,0.0,QMOULXS,IRUDRFB,8.0,ZVSTLNM,0.0,MHBRSQK
3,1,PSMFWTP,ZYAVJHP,None,15.0,ATQPZSJ,26,ZDTZNSB,THBWWCD,LTETYBG,...,IVIRTPR,GTISJWW,8640.0,0.0,IZLJUJS,IRUDRFB,14.0,ZBSRLCQ,0.0,GAZBSSZ
4,0,SLCRICD,QPQWGXA,None,13.0,CHZGJZR,20,PQZBVMG,MIGYEEG,LJBQPJW,...,NZGEZLW,WHSRKIM,41774.0,0.0,BHBIZCL,IRUDRFB,13.0,QHYLSBX,0.0,QTATWAY


결측치 처리

In [6]:
import numpy as np

# Function to fill NaNs based on existing distribution
def fill_na_with_distribution(column):
    counts = column.dropna().value_counts(normalize=True)
    nans_to_fill = column.isna().sum()
    fill_values = np.random.choice(counts.index, size=nans_to_fill, p=counts.values)
    column.loc[column.isna()] = fill_values
    return column

# Apply the function to each column in DataFrame
train = train.apply(fill_na_with_distribution, axis=0)
test = test.apply(fill_na_with_distribution, axis=0)

In [7]:
train = train.groupby('Click').apply(lambda x: x.sample(min(len(x), 5569860)))

In [8]:
train_x = train.drop('Click', axis = 1)
train_y = train['Click']

In [9]:
del train

In [10]:
# Label encode categorical columns
def label_encode_columns(train_df, test_df):
    le = LabelEncoder()
    for column in train_df.columns:
        if train_df[column].dtype == 'object' or isinstance(train_df[column].dtype, pd.CategoricalDtype):
            # Fit the LabelEncoder on the combined data to ensure consistency
            combined_data = pd.concat([train_df[column], test_df[column]], axis=0).astype(str)
            le.fit(combined_data)
            train_df[column] = le.transform(train_df[column].astype(str))
            test_df[column] = le.transform(test_df[column].astype(str))
    return train_df, test_df

# Apply label encoding to train_x and test_x
train_x, test_x = label_encode_columns(train_x, test)

In [12]:
train_x.to_parquet('/content/drive/MyDrive/data/train_v4.parquet',compression='gzip',engine='pyarrow',index=False)
test_x.to_parquet('/content/drive/MyDrive/data/test_v4.parquet',compression='gzip',engine='pyarrow',index=False)
train_y.to_csv('/content/drive/MyDrive/data/train_y_v4.csv',index = False)

In [11]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

Oputna + LGBM

In [13]:
pip install optuna

In [14]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from lightgbm import early_stopping

In [15]:
# Define the objective function
def objective(trial):
    # Define the search space for the hyperparameters
    param = {
        'objective': 'binary',
        'metric': 'auc',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        # 'num_leaves': trial.suggest_int('num_leaves', 100, 256),
        'max_depth': trial.suggest_int('max_depth',3, 100),
        'max_bin' : 1023,
        'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1e-1),
        'n_estimators': trial.suggest_int('n_estimators', 1,10000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        # 'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        # 'reg_alpha' : 0.04538159470655356,
        # 'reg_lambda':0.2207903936520441,
    }

    # Create and train the model
    model = lgb.LGBMClassifier(**param)
    # Use smaller subset of the data
    X_train_subset = X_train.sample(frac=0.3, random_state=42)
    y_train_subset = y_train[X_train_subset.index]
    X_val_subset = X_val.sample(frac=0.3, random_state=42)
    y_val_subset = y_val[X_val_subset.index]
    model.fit(X_train_subset, y_train_subset, eval_set=[(X_val_subset, y_val_subset)], callbacks=[early_stopping(stopping_rounds=10)])

    # Predict and calculate AUC
    y_pred = model.predict_proba(X_val_subset)[:, 1]
    auc = roc_auc_score(y_val_subset, y_pred)

    return auc

In [ ]:
# Create a study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-05-31 12:44:49,373] A new study created in memory with name: no-name-53e8c31f-8b9c-477c-a082-b046066cdb1e


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[3116]	valid_0's auc: 0.761607


[I 2024-05-31 12:54:11,489] Trial 0 finished with value: 0.7616072588512309 and parameters: {'max_depth': 66, 'learning_rate': 0.05427277185147264, 'n_estimators': 3668, 'min_child_samples': 70, 'subsample': 0.4117868967557664}. Best is trial 0 with value: 0.7616072588512309.


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[3495]	valid_0's auc: 0.762906


[I 2024-05-31 13:04:13,813] Trial 1 finished with value: 0.7629058903964258 and parameters: {'max_depth': 68, 'learning_rate': 0.06268725373548048, 'n_estimators': 8069, 'min_child_samples': 77, 'subsample': 0.42190194310241425}. Best is trial 1 with value: 0.7629058903964258.


...

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1616]	valid_0's auc: 0.760963


[I 2024-05-31 14:24:57,136] Trial 11 finished with value: 0.7609633670732038 and parameters: {'max_depth': 48, 'learning_rate': 0.09814780337542162, 'n_estimators': 9987, 'min_child_samples': 65, 'subsample': 0.5368771957398967}. Best is trial 1 with value: 0.7629058903964258.


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1307]	valid_0's auc: 0.759113


[I 2024-05-31 14:29:04,424] Trial 12 finished with value: 0.7591126801980568 and parameters: {'max_depth': 66, 'learning_rate': 0.09268268040827357, 'n_estimators': 7079, 'min_child_samples': 57, 'subsample': 0.5291105324025369}. Best is trial 1 with value: 0.7629058903964258.


Training until validation scores don't improve for 10 rounds


In [ ]:
best_params = study.best_trial.params
best_model = lgb.LGBMClassifier(**best_params)

In [ ]:
sample_submission = pd.read_csv('drive/MyDrive/data/sample_submission.csv')

# Predict probabilities on the test set
y_pred_test_proba = best_model.predict_proba(test_x)[:, 1]

# Output the probabilities for test set
print(y_pred_test_proba)

In [ ]:
sample_submission['Click'] = y_pred_test_proba
sample_submission

In [ ]:
sample_submission.to_csv("drive/MyDrive/data/lgbm_prediction_v5.csv",index = False)

모델링

In [ ]:
!pip install flaml

In [ ]:
from flaml import AutoML
# import wandb

In [ ]:
# Initialize AutoML
automl = AutoML()

automl_settings = {
    "time_budget": 3600,  # Total time budget in seconds
    "metric": 'roc_auc',  # Evaluation metric
    "task": 'classification',  # Task type
    "log_file_name": 'automl.log',  # Log file
    #"estimator_list": ['lgbm', 'xgboost', 'catboost', 'rf', 'extra_tree'],  # List of estimators to use
     "estimator_list": ['lgbm'],
    #"eval_method": "holdout",  # Use holdout validation method
    # "split_ratio": 0.2,  # Ratio of data to be used as validation set
    "early_stop": 10
}


In [ ]:
# Fit the model
automl.fit(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, **automl_settings)

[flaml.automl.logger: 05-31 00:53:58] {1680} INFO - task = classification
[flaml.automl.logger: 05-31 00:53:58] {1688} INFO - Data split method: stratified
[flaml.automl.logger: 05-31 00:53:58] {1691} INFO - Evaluation method: holdout
[flaml.automl.logger: 05-31 00:54:08] {1789} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl.logger: 05-31 00:54:08] {1901} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl.logger: 05-31 00:54:08] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 05-31 00:54:09] {2345} INFO - Estimated sufficient time budget=9354179s. Estimated necessary time budget=9354s.
[flaml.automl.logger: 05-31 00:54:09] {2392} INFO -  at 81.8s,	estimator lgbm's best error=0.3585,	best estimator lgbm's best error=0.3585
[flaml.automl.logger: 05-31 00:54:09] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 05-31 00:54:10] {2392} INFO -  at 82.7s,	estimator lgbm's best error=0.3585,	best estimator lgbm's best error=0.358

In [ ]:
sample_submission = pd.read_csv('drive/MyDrive/data/sample_submission.csv')

# Predict probabilities on the test set
y_pred_test_proba = automl.predict_proba(test_x)[:, 1]

# Output the probabilities for test set
print(y_pred_test_proba)

[0.51818208 0.4127019  0.35411755 ... 0.19848276 0.52919502 0.66582816]


In [ ]:
sample_submission['Click'] = y_pred_test_proba
sample_submission

,ID,Click
0,TEST_0000000,0.518182
1,TEST_0000001,0.412702
2,TEST_0000002,0.354118
3,TEST_0000003,0.713257
4,TEST_0000004,0.716708
...,...,...
4538536,TEST_4538536,0.557996
4538537,TEST_4538537,0.639353
4538538,TEST_4538538,0.198483
4538539,TEST_4538539,0.529195


In [ ]:
sample_submission.to_csv("drive/MyDrive/data/lgbm_prediction_v4.csv",index = False)